# OpenAI Chat Completions API
https://platform.openai.com/docs/overview  
https://platform.openai.com/docs/api-reference/chat  

배포된 openai의 api key를 .env의 OPENAI_API_KEY에 등록하여 사용합니다.

In [1]:
import requests
from pprint import pprint
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
URL = "https://api.openai.com/v1/chat/completions"
model = "gpt-4o-mini"


### REST API 요청
라이브러리 없이 직접 HTTP 통신을 통해 api를 호출한다.

In [2]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {OPENAI_API_KEY}"
}

payload = {
    "model": model,
    "messages": [
        {"role": "system", "content": "당신은 친절한 AI 강사입니다."},
        {"role": "user", "content": "Chat Completions API가 뭐야? 2~3문장으로 답변해줘"}
    ]
}

response = requests.post(URL, headers=headers, json=payload)
pprint(response.json())
print(response.json()['choices'][0]['message']['content'])

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'annotations': [],
                          'content': 'Chat Completions API는 사용자가 입력한 메시지에 대한 '
                                     '응답을 생성하는 AI 모델을 제공하는 인터페이스입니다. 이 API는 '
                                     '자연어 처리 기술을 활용하여 대화형 응답을 생성하고, 다양한 상황에 맞춰 '
                                     '사용자와 상호작용할 수 있도록 돕습니다. 이를 통해 챗봇, 고객 지원, '
                                     '개인비서 등 다양한 애플리케이션에 적용할 수 있습니다.',
                          'refusal': None,
                          'role': 'assistant'}}],
 'created': 1768959923,
 'id': 'chatcmpl-D0HZbOQyjocIXJ5KH27055F4ORChm',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_29330a9688',
 'usage': {'completion_tokens': 92,
           'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                         'audio_tokens': 0,

### OpenAI SDK를 활용한 요청
공식 라이브러리를 사용하여 생산성을 높이는 표준 방식이다.  
`pip install openai` 를 통해 설치한다.

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "user", "content": "Openai SDK를 사용하면 어떤 점이 좋아?"}
    ]
)

print(completion.choices[0].message.content)

OpenAI SDK를 사용하면 여러 가지 이점이 있습니다:

1. **간편한 통합**: OpenAI의 API는 다양한 프로그래밍 언어와 플랫폼에서 쉽게 통합할 수 있도록 설계되어 있어, 개발자가 빠르게 기능을 구현할 수 있습니다.

2. **강력한 모델**: ChatGPT와 같은 고급 AI 모델을 통해 자연어 처리(NLP) 작업을 수행할 수 있습니다. 이는 텍스트 생성, 질문 응답, 요약, 번역 등 다양한 응용 프로그램에 활용될 수 있습니다.

3. **사용자 맞춤화**: OpenAI SDK를 사용하면 특정 용도에 맞도록 모델의 응답을 조정하고, 이를 통해 보다 개인화된 사용자 경험을 제공할 수 있습니다.

4. **지속적인 업데이트**: OpenAI는 모델을 지속적으로 개선하고 업데이트하므로, 최신 기술을 바탕으로 새로운 기능과 성능 개선을 누릴 수 있습니다.

5. **합리적인 비용**: API 사용량에 따라 요금이 부과되기 때문에, 소규모 프로젝트부터 대규모 프로젝트까지 다양한 예산에 맞춰 활용 가능합니다.

6. **활발한 커뮤니티와 지원**: OpenAI의 SDK는 개발자 커뮤니티가 활발하여, 문제 해결이나 사용 사례 관련 도움을 받을 수 있는 리소스가 많습니다.

7. **다양한 기능**: 텍스트 뿐만 아니라 코드 생성, 이미지 생성 등 다양한 AI 기능을 사용할 수 있어, 창의적인 프로젝트에 유용합니다.

이러한 이점들 덕분에 OpenAI SDK는 다양한 산업에서 널리 사용되고 있습니다.


### System Prompt 비교

동일한 질문에 대해 AI의 페르소나에 따라 답변이 어떻게 달라지는지 확인해 보자

In [4]:
user_input = "아침 일찍 일어나는 습관의 장점에 대해 말해줘."

personas = {
    "열정적인 셰프": "당신은 요리에 인생을 건 셰프입니다. 인생의 모든 이치를 요리 과정과 재료에 비유하여 설명하세요.",
    "엄격한 헬스 트레이너": "당신은 매우 엄격한 운동 전문가입니다. 강한 어조로 자기관리를 강조하며 답변하세요.",
    "지혜로운 판다": "당신은 대나무 숲에 사는 느긋하고 지혜로운 판다입니다. 느릿느릿하고 평화로운 말투로 조언을 건네세요."
}

for name, prompt in personas.items():
    print(f"--- [{name}] 버전 ---")
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_input}
        ]
    )
    print(response.choices[0].message.content)
    print("\n")

--- [열정적인 셰프] 버전 ---
아침 일찍 일어나는 습관을 요리 과정에 비유해 보겠습니다.

아침 일찍 일어난다는 것은 마치 좋은 레시피를 준비하는 것과 같습니다. 요리를 시작하려면, 먼저 재료를 손질하고, 쿠킹 스튜디오를 정리해야 하죠. 이렇게 아침 일찍 일어나 주어진 하루라는 재료를 계획하고 준비하는 시간이 주어집니다. 

1. **신선한 재료를 활용하기** - 아침에 일찍 일어나면 하루의 시작을 신선하게 맞이할 수 있습니다. 마치 신선한 채소와 향신료를 사용하는 것처럼, 아침의 차분한 공기는 마음을 정리하고, 더 나은 결정을 내리는 데 도움을 줍니다.

2. **조리 시간을 확보하기** - 요리를 하려면 충분한 시간이 필요하듯이, 아침 일찍 일어나는 것은 하루를 계획할 수 있는 ‘조리 시간’을 확보하는 것입니다. 이 시간을 통해 운동을 하거나 독서를 하면서 자신에게 투자할 수 있습니다.

3. **일관된 맛을 만들어내다** - 요리에서 일관된 맛은 실패를 줄이는 중요한 요소입니다. 아침 일찍 일어나는 일관된 습관은 여러분의 생체 리듬을 규칙적으로 만들어, 하루의 생산성을 높이고 스트레스를 감소시킵니다.

4. **다양한 요리법 도전하기** - 일찍 일어나 여유로운 아침을 가지면, 새로운 요리법에 도전할 시간이 생깁니다. 마찬가지로, 이것은 새로운 취미나 업무를 시도할 수 있는 기회를 제공합니다.

5. **완벽한 플레이트와 마무리** - 요리의 마지막 단계는 요리를 서빙하는 것입니다. 아침을 일찍 시작하면 여러분의 하루를 마무리할 시간을 갖고, 하루에 이루고자 했던 목표를 돌아보고 반성할 수 있는 시간을 가질 수 있습니다. 

아침 일찍 일어나는 것은 결국 삶이라는 요리를 좀 더 풍요롭고 맛있게 만드는 비법입니다. 일상의 모든 과정을 소중히 여기고, 매일매일 최고의 요리를 만들어 나가는 태도를 가지는 것! 그것이 진정한 셰프의 자세라고 할 수 있습니다.


--- [엄격한 헬스 트레이너] 버전 ---
아침 일찍 일어나는 습관은 절대적으로 필수적입니다

### Temperature 비교

동일한 질문에 대해 temperature에 따라 답변이 어떻게 달라지는지 확인해 보자

In [ ]:
creative_topic = "운동화 브랜드의 새로운 슬로건을 5개 제안해줘. 단, '속도'나 '승리' 같은 뻔한 단어는 제외하고 아주 기발하게 작성해줘."
temperatures = [0.3, 0.8, 1.0, 1.3, 1.5, 1.6, 1.8]

for t in temperatures:
    print(f"### 설정값 (Temperature): {t} ###")
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": creative_topic}],
        temperature=t,                  #temperature 설정
        max_completion_tokens=200, 
        timeout=15.0
    )
    print(response.choices[0].message.content)
    print("=" * 50)

In [ ]:
creative_topic = "우리집 강아지의 별명을 3개 지어줘."
temperatures = [0.3, 0.8, 1.0, 1.3, 1.5, 1.6, 1.8]

for t in temperatures:
    print(f"### 설정값 (Temperature): {t} ###")
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": creative_topic}],
        temperature=t,
        max_completion_tokens=200, 
        timeout=15.0
    )
    print(response.choices[0].message.content)
    print("=" * 50)

### `messages` 배열을 활용한 대화 맥락 유지 (Context Window)
Chat Completions API는 상태를 저장하지 않는(Stateless) 방식이므로, 이전 대화 내역을 리스트에 계속 누적해서 보내야 한다.

In [5]:
def chat_without_memory(user_input):
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": user_input}
        ]
    )
    
    # 3. 모델의 답변을 기록에 추가 (이것이 맥락 유지의 핵심)
    answer = response.choices[0].message.content
    
    return answer

# 실습 테스트
print("Q1: 내 이름은 jun이야.")
print(f"A1: {chat_without_memory('내 이름은 jun이야')}\n")

print("Q2: 내 이름이 뭐라고?")
print(f"A2: {chat_without_memory('내 이름이 뭐라고?')}")

Q1: 내 이름은 jun이야.
A1: 안녕하세요, Jun! 무엇을 도와드릴까요?

Q2: 내 이름이 뭐라고?
A2: 죄송하지만, 당신의 이름을 알 수 있는 정보는 없습니다. 당신의 이름을 알려주시면 그에 맞춰 대화할 수 있습니다!


In [ ]:
# 대화 내역을 저장할 리스트 초기화
history = [
    {"role": "system", "content": "당신은 사용자의 이름을 기억하는 비서입니다."}
]
def chat_with_memory(user_input):
    # 1. 사용자 질문을 기록에 추가
    history.append({"role": "user", "content": user_input})
    
    # 2. 전체 기록을 API에 전송
    response = client.chat.completions.create(
        model=model,
        messages=history
    )
    
    # 3. 모델의 답변을 기록에 추가 (이것이 맥락 유지의 핵심)
    answer = response.choices[0].message.content
    history.append({"role": "assistant", "content": answer})
    
    return answer

# 실습 테스트
print("Q1: 내 이름은 jun이야.")
print(f"A1: {chat_with_memory('내 이름은 jun이야.')}\n")

print("Q2: 내 이름이 뭐라고?")
print(f"A2: {chat_with_memory('내 이름이 뭐라고?')}")

Q1: 내 이름은 jun이야.
A1: 만나서 반가워, Jun! 어떻게 도와드릴까요?

Q2: 내 이름이 뭐라고?
A2: 당신의 이름은 Jun입니다!


### Structured Outputs (구조화된 출력)
모델의 답변을 단순히 텍스트로 받는 것이 아니라, JSON 형태로 고정하여 받을 수 있다.  
웹 서비스의 백엔드에서 데이터를 바로 처리해야 할 때 필수적인 기능이다.  
여기서는 `JSON mode(json_object)`로 json format을 활용하지만,  
이후에는 pydantic 라이브러리를 활용한 `JSON Scheme` 방식을 통해 명확한 json 응답 형식을 지정한다.

In [7]:
import json

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "너는 요리사야. 답변은 반드시 JSON 형식으로 해줘."},
        {"role": "user", "content": "떡볶이 레시피 알려줘."}
    ],
    # JSON 모드 활성화
    response_format={"type": "json_object"}
)

# 문자열로 온 답변을 직접 파싱해야 함
res_json = json.loads(response.choices[0].message.content)
print(res_json)

{'recipe': {'name': '떡볶이', 'ingredients': [{'item': '떡', 'quantity': '300g'}, {'item': '어묵', 'quantity': '150g'}, {'item': '대파', 'quantity': '1대'}, {'item': '양배추', 'quantity': '100g'}, {'item': '고추장', 'quantity': '2큰술'}, {'item': '고춧가루', 'quantity': '1큰술'}, {'item': '간장', 'quantity': '1큰술'}, {'item': '설탕', 'quantity': '1큰술'}, {'item': '물', 'quantity': '500ml'}, {'item': '깨소금', 'quantity': '약간'}], 'instructions': ['1. 떡은 물에 담가두어 부드럽게 만든다.', '2. 대파는 어슷하게 썰고, 양배추는 적당한 크기로 자른다.', '3. 냄비에 물을 붓고 떡과 어묵을 넣어 끓인다.', '4. 떡이 익으면 고추장, 고춧가루, 간장, 설탕을 넣어 잘 저어준다.', '5. 대파와 양배추를 넣고 5분 정도 더 끓인다.', '6. 불을 끄고 깨소금을 뿌려 마무리한다.', '7. 뜨겁게 해서 접시에 담아 맛있게 즐긴다.'], 'serving': '2인분'}}


### Streaming (실시간 응답 처리)
stream=True 설정을 통해 활성화한다.  
서버는 SSE(Server-Sent Events) 프로토콜을 사용하여 응답을 끊지 않고 조각(Chunk) 단위로 지속적으로 전송한다.  
응답 객체는 제너레이터 형식으로, for 루프를 사용해 활용할 수 있다.

In [ ]:
prompt = "양자 역학에 대해 초등학생도 이해할 수 있게 설명해줘."
print(f"질문: {prompt}\n")
print("답변: ", end="")

response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt}],
    stream=True   ####
)

full_response = ""
for chunk in response:
    content = chunk.choices[0].delta.content
    if content:``
        print(content, end="", flush=True) # flush 옵션을 통해 출력 버퍼를 즉시 비워 스트리밍 답변이 지연 없이 실시간으로 표시되도록 한다.
        full_response += content

print("\n\n--- 스트리밍 종료 ---")

질문: 양자 역학에 대해 초등학생도 이해할 수 있게 설명해줘.

답변: 양자 역학은 매우 작은 것들이 어떻게 행동하는지를 연구하는 과학의 한 분야예요. 우리가 주변에서 보는 물체들, 예를 들어 공기 중의 먼지나 컴퓨터의 부품들은 모두 아주 작은 입자로 이루어져 있어요. 이 입자들은 원자라고 부르는데, 원자는 다시 더 작은 입자들인 전자, 양성자, 중성자로 이루어져 있어요.

그런데 양자 역학에서는 이 아주 작은 입자들이 우리가 생각하는 것과는 다른 방식으로 움직이고 행동해요. 예를 들어, 원자 안의 전자는 우리가 생각하는 것처럼 정확한 위치에 있지 않아요. 오히려 전자는 특정한 공간에서 '확률'로 존재하는데, 이걸 비유하자면 숨바꼭질을 하고 있는 아이와 같아요. 우리가 아이가 숨은 곳을 모르듯이, 전자도 정확한 위치를 알 수 없어요. 대신, 전자가 있을 만한 여러 장소가 있어요.

또한, 양자 역학에서는 입자들이 동시에 여러 상태일 수 있는 "슈뢰딩거의 고양이"라는 유명한 이야기가 있어요. 상자 안에 있는 고양이가 살아있기도 하고 죽어있기도 하다는 이 아이디어는, 아주 작은 입자들이 우리가 이해하는 시간과 공간의 법칙과는 다르게 행동할 수 있다는 것을 보여줘요.

쉽게 말해, 양자 역학은 우리가 일상에서 경험하는 것과는 전혀 다른 규칙들이 지배하는 아주 작은 세상을 설명하는 과학이란 거예요!

--- 스트리밍 종료 ---


### 비동기 요청


In [ ]:
from openai import AsyncOpenAI
import asyncio

async_client = AsyncOpenAI(api_key=OPENAI_API_KEY)

async def get_food_recommendation(city):
    print(f"[{city}] 맛집 검색 시작...")
    response = await async_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": f"{city}에 가면 꼭 먹어야 할 음식 딱 한 가지만 추천해줘."}]
    )
    print(f"[{city}] 검색 완료!")
    return f"{city}: {response.choices[0].message.content}"

async def main():
    cities = ["서울", "파리", "뉴욕", "도쿄", "방콕", "로마"]
    tasks = [get_food_recommendation(c) for c in cities]
    
    # 여러 요청을 동시에(병렬로) 처리
    results = await asyncio.gather(*tasks)
    
    print("\n--- [여행자들을 위한 미식 가이드] ---")
    for r in results:
        print(r)

await main()

### Logprobs - 확률 확인하기

In [ ]:
import math

prompt = "새로 오픈한 조용한 북카페 이름을 한글로 딱 하나만 추천해줘."
response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt}],
    logprobs=True,
    top_logprobs=3,
    max_completion_tokens=50
)

content = response.choices[0].message.content
logprobs_data = response.choices[0].logprobs.content

print(f"질문: {prompt}")
print(f"답변: {content}\n")
print(f"{'Token':<15} | {'Probability':<12} | {'Top Alternatives'}")
print("-" * 60)

for lp in logprobs_data:
    prob = math.exp(lp.logprob) * 100
    alternatives = [f"{top.token}({math.exp(top.logprob)*100:.1f}%)" for top in lp.top_logprobs]
    print(f"{lp.token:<15} | {prob:>10.2f}% | {', '.join(alternatives)}")

